In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import string

In [2]:
class NameModel(nn.Module):
    def __init__(self, input_size, hidden_size, number_of_layers, output_size, number_of_chars, batch_size):
        super(NameModel, self).__init__()
        self.number_of_chars = number_of_chars
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.number_of_layers = number_of_layers
        self.batch_size = batch_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=number_of_layers, batch_first=True)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.fc3 = nn.Linear(output_size, output_size)
    def forward(self, X, states):
        h, c = states
        out, (h, c) = self.lstm(X, (h, c))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out, (h, c)
    def initStateArguments(self):
        h = torch.zeros((self.number_of_layers, self.batch_size, self.hidden_size))
        c = torch.zeros((self.number_of_layers, self.batch_size, self.hidden_size))
        return h, c

In [3]:
def generateNames(model, start, no_of_chars, number_of_names):
    maximum = 5
    number_of_layers = 1
    hidden_size = 27
    names = []
    for i in range(number_of_names):
        char = start
        name = start
        h = torch.zeros((number_of_layers, 1, hidden_size))
        t = torch.zeros((number_of_layers, 1, hidden_size))
        chosen_index = np.inf
        length = 1
        previous_length = no_of_chars
        counter = 1
        while length < no_of_chars and chosen_index != 26:
            X = torch.zeros((1, 1, 27))
            X[0, 0, alphabet_encoding[char]] = 1
            out, (h, t) = model(X, (h, t))
            val, ind = torch.topk(out[0][0], maximum)
            index_to_change = np.random.randint(1, previous_length)
            if length == counter:
                chosen_index = np.random.randint(1, maximum)
                char = id_to_char[ind[chosen_index].item()]
            else:
                chosen_index = ind[0].item()
                char = id_to_char[chosen_index]
            name += char
            length += 1
            counter += 1
            counter = counter % 5
        previous_length = length
        names.append(name)
    return names

In [4]:
alphabet_encoding = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12,
                    'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24,
                    'z': 25, '\n': 26}
chars = string.ascii_lowercase + '\n'
char_to_id = {c:i for i, c in enumerate(chars)}
id_to_char = {v:k for k, v in char_to_id.items()}

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NameModel(27, 27, 1, 27, 11, 39)
model.load_state_dict(torch.load(f"../input/namesgeneratormodel/modelLSTM2.pth"))
model.eval()

NameModel(
  (lstm): LSTM(27, 27, batch_first=True)
  (fc2): Linear(in_features=27, out_features=27, bias=True)
  (fc3): Linear(in_features=27, out_features=27, bias=True)
)

In [6]:
alphabets = string.ascii_lowercase
letter_index = np.random.randint(0, len(alphabets) - 1)
letter = alphabets[letter_index]
#letter = input("Enter a letter")
names = generateNames(model, letter, 11, 20)
output_names = []
for i in range(len(names)):
    output_names.append(names[i].replace('\n', ''))
    print(output_names[i])

noher
nohem
nislo
neslow
neevia
noavalena
nossa
nissia
nysten
nistian
nilly
noadamari
niyula
nycky
nysan
noahan
neeve
nycille
neiar
neison
